In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import datetime as dt
import plotly.graph_objects as go


import os
import pickle
from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
import scripts.functions as fn 

from scripts.indicators import resample, join_after_resample, HeikinAshi

from binance.client import Client

#Conexion con Binance sin API-KEY
client = Client()

c:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app\backtest


In [2]:
#Funcion para descarga de velas
def get_klines(symbol,interval,start,end):
    print(symbol, interval, start, end )
    df = client.get_historical_klines(symbol=symbol, interval=interval, start_str=start, end_str=end)
    df = pd.DataFrame(df)
    df = df.iloc[:, :6]
    df.columns = ["datetime", "open", "high", "low", "close", "volume"]
    df['open'] = df['open'].astype('float')
    df['high'] = df['high'].astype('float')
    df['low'] = df['low'].astype('float')
    df['close'] = df['close'].astype('float')
    df['volume'] = df['volume'].astype('float')
    df['datetime'] = pd.to_datetime(df['datetime'], unit='ms') - dt.timedelta(hours=3)
    #df=df.set_index('datetime')
    return df

In [25]:
klines_file  = './backtest/klines/0m01/Completo_BTCUSDT_0m01_2021-01-01_2023-07-31.DataFrame'
with open(klines_file, 'rb') as file:
    df = pickle.load(file)
    #df.set_index('datetime',inplace=True)

df = resample(df,(60*24))

#start = '2024-05-01'
#end = '2024-11-15'
#symbol = 'BTCUSDT'
#interval = '1d'
#real_start = (dt.datetime.strptime(start, "%Y-%m-%d") ).strftime("%Y-%m-%d")
#real_end = (dt.datetime.strptime(end, "%Y-%m-%d") ).strftime("%Y-%m-%d")
#utc_start = real_start+' 00:00:00'
#utc_end = real_end+' 23:59:59'
#df = get_klines(symbol=symbol,interval=interval, start=utc_start,end=utc_end)
##df = fn.ohlc_mirror_v(df)

#Filtrar rango de fechas
#ini = pd.to_datetime('2018-06-26')
#fin = pd.to_datetime('2028-05-01')
#df = df[(df['datetime']>=ini)&(df['datetime']<=fin)]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)


#df.drop(columns=['volume'],inplace=True)

#Quitar parte del dataframe
#df = df[500:1500]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)
df = HeikinAshi(df)

In [26]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import plotly.graph_objects as go

# Supongamos que ya tienes un DataFrame 'df' con las columnas mencionadas:
# ['open', 'high', 'low', 'close', 'volume', 'HA_open', 'HA_high', 'HA_low', 'HA_close']

# 1. **Preprocesamiento**
sequence_length = 50  # Longitud de la secuencia para el LSTM

# Calcular características adicionales
df['change_pct'] = (df['close'] - df['close'].shift(1)) / df['close'].shift(1) * 100
df['HA_body_size'] = abs(df['HA_close'] - df['HA_open'])
df['HA_shadow_size'] = df['HA_high'] - df['HA_low']

# Reemplazar valores NaN por 0
df.fillna(0, inplace=True)

# Seleccionar características para entrenamiento
features = ['change_pct', 'HA_body_size', 'HA_shadow_size']
scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

In [27]:
# 2. **Entrenamiento del modelo LSTM**
# Crear secuencias para entrenamiento
X, y = [], []

for i in range(sequence_length, len(df)):
    X.append(df[features].iloc[i-sequence_length:i].values)
    y.append(df['change_pct'].iloc[i])

X, y = np.array(X), np.array(y)

# Dividir datos en entrenamiento y prueba
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Crear el modelo LSTM
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(sequence_length, len(features))),
    LSTM(50, return_sequences=False),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy')
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
28/28 [==============================] - 4s 48ms/step - loss: 0.6877 - val_loss: 0.6879
Epoch 2/10
28/28 [==============================] - 1s 27ms/step - loss: 0.6872 - val_loss: 0.6878
Epoch 3/10
28/28 [==============================] - 1s 27ms/step - loss: 0.6872 - val_loss: 0.6879
Epoch 4/10
28/28 [==============================] - 1s 28ms/step - loss: 0.6871 - val_loss: 0.6878
Epoch 5/10
28/28 [==============================] - 1s 29ms/step - loss: 0.6870 - val_loss: 0.6878
Epoch 6/10
28/28 [==============================] - 1s 31ms/step - loss: 0.6872 - val_loss: 0.6881
Epoch 7/10
28/28 [==============================] - 1s 29ms/step - loss: 0.6871 - val_loss: 0.6878
Epoch 8/10
28/28 [==============================] - 1s 29ms/step - loss: 0.6870 - val_loss: 0.6879
Epoch 9/10
28/28 [==============================] - 1s 28ms/step - loss: 0.6871 - val_loss: 0.6880
Epoch 10/10
28/28 [==============================] - 1s 27ms/step - loss: 0.6871 - val_loss: 0.6879


In [28]:
# 3. **Generar probabilidades de corrección**
probabilities = []

for i in range(len(df)):
    if i >= sequence_length:
        sequence = df[features].iloc[i-sequence_length:i].values
        sequence = np.expand_dims(sequence, axis=0)
        probabilities.append(model.predict(sequence)[0, 0])
    else:
        probabilities.append(np.nan)

df['correction_probability'] = probabilities

1/1 [==============================] - 0s 23ms/step


In [29]:
# 4. **Graficar resultados con Plotly**
# Crear un gráfico interactivo
fig = go.Figure()

# Línea de precios
fig.add_trace(go.Scatter(
    x=df.index,
    y=df['close'],
    mode='lines',
    name='Precio de cierre',
    line=dict(color='blue')
))

# Correcciones detectadas
corrections = df[df['correction_probability'] > 0.43]

fig.add_trace(go.Scatter(
    x=corrections.index,
    y=corrections['close'],
    mode='markers',
    name='Correcciones detectadas',
    marker=dict(color='red', size=8, symbol='circle')
))

# Configurar layout del gráfico
fig.update_layout(
    title='Correcciones detectadas por el modelo',
    xaxis_title='Tiempo',
    yaxis_title='Precio',
    legend=dict(orientation='h', x=0.42, xanchor='center'),
    template='plotly_dark'
)

fig.show()


In [30]:
df['correction_probability'].describe()


count    1102.000000
mean        0.442443
std         0.003041
min         0.432990
25%         0.440114
50%         0.443198
75%         0.444989
max         0.446815
Name: correction_probability, dtype: float64

In [31]:
df_to_show = df #[0:20]
#df_to_show['corrections'] = corrections
"""
indicators = [
     #{'col': 'renko_trend_up','name': 'renko_trend_up','color': 'green','row': 1, 'mode':'markers',},
     #{'col': 'renko_trend_down','name': 'renko_trend_down','color': 'red','row': 1, 'mode':'markers',},
     #{'col': 'trend-chg','name': 'trend-chg','color': 'yellow','row': 1, 'mode':'markers',},
     #{'col': 'st_high','name': 'ST High','color': 'green','row': 1, 'mode':'markers',},
     #   {'col': 'FL_0.0%', 'name': 'FL_0.0%', 'color': 'red', 'row': 1, 'mode': 'lines', }, 
     #   {'col': 'FL_gold1', 'name': 'FL_gold1', 'color': 'orange', 'row': 1, 'mode': 'lines', }, 
     #   {'col': 'FL_gold2', 'name': 'FL_gold2', 'color': 'orange', 'row': 1, 'mode': 'lines', }, 
     #   {'col': 'FL_100.0%', 'name': 'FL_100.0%', 'color': 'green', 'row': 1, 'mode': 'lines', }, 
        ]

indicators_out = [
     #{'col': 'pred','name': 'pred','color': 'yellow','row': 1, 'mode':'lines',},
     #{'col': 'VOL_perc','name': 'Volume_SMA','color': 'green','row': 1, 'mode':'lines',},
        ]

#https://plotly.com/python/reference/scatter/#scatter-marker-symbol
events = [
       {'df':df_to_show,'col':'corrections','name': 'corrections','color': 'green','symbol': 'circle' }, 
       
     ]


fig = fn.ohlc_chart(df_to_show,indicators=indicators,indicators_out=indicators_out,events=events,) 
fig.show()
"""
corrections.tail()

,datetime,open,high,low,close,volume,HA_close,HA_open,HA_high,HA_low,HA_side,HA_low_trend,change_pct,HA_body_size,HA_shadow_size,correction_probability
1147,2023-07-27,29351.95,29567.49,29083.85,29222.78,22476.47626,29306.5175,29316.19375,29567.49,29083.85,0,1,0.427877,0.001779,0.029132,0.445881
1148,2023-07-28,29222.78,29542.22,29123.12,29314.14,23993.61627,29300.5650,29296.94125,29542.22,29123.12,0,1,0.449435,0.000635,0.024349,0.445898
1149,2023-07-29,29314.14,29406.92,29256.18,29352.90,10851.36844,29332.5350,29284.51250,29406.92,29256.18,1,0,0.444268,0.009030,0.004463,0.445921
1150,2023-07-30,29352.90,29449.00,29033.24,29281.09,15706.97441,29279.0575,29333.02750,29449.00,29033.24,-1,0,0.433475,0.010155,0.024102,0.445998
1151,2023-07-31,29281.09,29530.00,29101.80,29232.25,22605.48964,29286.2850,29298.02625,29530.00,29101.80,0,1,0.435704,0.002170,0.025024,0.446080
